# Food Delivery Data Hackathon
## Dataset Merging and Analysis

---
## Part 1: Merging Datasets

In [16]:
# Import required libraries
import pandas as pd
import json
import re

In [17]:
# Step 1: Load CSV Data (Orders)
print("Step 1: Loading orders.csv...")
orders_df = pd.read_csv('orders.csv')
print(f"  Orders loaded: {len(orders_df)} records")
print(f"  Columns: {list(orders_df.columns)}")
orders_df.head()

Step 1: Loading orders.csv...
  Orders loaded: 10000 records
  Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [18]:
# Step 2: Load JSON Data (Users)
print("Step 2: Loading users.json...")
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)
print(f"  Users loaded: {len(users_df)} records")
print(f"  Columns: {list(users_df.columns)}")
users_df.head()

Step 2: Loading users.json...
  Users loaded: 3000 records
  Columns: ['user_id', 'name', 'city', 'membership']


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [19]:
# Step 3: Load SQL Data (Restaurants)
print("Step 3: Loading restaurants.sql...")
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()

# Parse INSERT statements to extract restaurant data
pattern = r"INSERT INTO restaurants VALUES \((\d+), '([^']+)', '([^']+)', ([\d.]+)\);"
matches = re.findall(pattern, sql_content)

restaurants_data = []
for match in matches:
    restaurants_data.append({
        'restaurant_id': int(match[0]),
        'restaurant_name_sql': match[1],
        'cuisine': match[2],
        'rating': float(match[3])
    })

restaurants_df = pd.DataFrame(restaurants_data)
print(f"  Restaurants loaded: {len(restaurants_df)} records")
print(f"  Columns: {list(restaurants_df.columns)}")
restaurants_df.head()

Step 3: Loading restaurants.sql...
  Restaurants loaded: 500 records
  Columns: ['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating']


,restaurant_id,restaurant_name_sql,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [20]:
# Step 4: Merge the Data using Left Joins
print("Step 4: Merging datasets...")

# First merge: orders with users (on user_id)
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')
print(f"  After merging with users: {len(merged_df)} records")

# Second merge: result with restaurants (on restaurant_id)
merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left')
print(f"  After merging with restaurants: {len(merged_df)} records")

Step 4: Merging datasets...
  After merging with users: 10000 records
  After merging with restaurants: 10000 records


In [21]:
# Step 5: Create Final Dataset
print("Step 5: Creating final dataset...")

# Rename columns for clarity
final_df = merged_df.rename(columns={
    'name': 'user_name',
    'restaurant_name': 'restaurant_name_orders'
})

# Select and reorder columns for the final output
final_columns = [
    'order_id', 'user_id', 'user_name', 'city', 'membership',
    'restaurant_id', 'restaurant_name_orders', 'cuisine', 'rating',
    'order_date', 'total_amount'
]

# Check which columns exist
available_columns = [col for col in final_columns if col in final_df.columns]
final_df = final_df[available_columns]

# Save to CSV
output_file = 'final_food_delivery_dataset.csv'
final_df.to_csv(output_file, index=False)
print(f"\nFinal dataset saved to: {output_file}")
print(f"Total records: {len(final_df)}")
print(f"Columns: {list(final_df.columns)}")
final_df.head()

Step 5: Creating final dataset...

Final dataset saved to: final_food_delivery_dataset.csv
Total records: 10000
Columns: ['order_id', 'user_id', 'user_name', 'city', 'membership', 'restaurant_id', 'restaurant_name_orders', 'cuisine', 'rating', 'order_date', 'total_amount']


,order_id,user_id,user_name,city,membership,restaurant_id,restaurant_name_orders,cuisine,rating,order_date,total_amount
0,1,2508,User_2508,Hyderabad,Regular,450,New Foods Chinese,Mexican,3.2,18-02-2023,842.97
1,2,2693,User_2693,Pune,Regular,309,Ruchi Curry House Multicuisine,Indian,4.5,18-01-2023,546.68
2,3,2084,User_2084,Chennai,Gold,107,Spice Kitchen Punjabi,Mexican,4.0,15-07-2023,163.93
3,4,319,User_319,Bangalore,Gold,224,Darbar Kitchen Non-Veg,Chinese,4.8,04-10-2023,1155.97
4,5,1064,User_1064,Pune,Regular,293,Royal Eatery South Indian,Italian,3.0,25-12-2023,1321.91


---
## Part 2: Load Final Dataset for Analysis

In [22]:
# Load the final dataset
df = pd.read_csv('final_food_delivery_dataset.csv')
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')
print(f"Dataset loaded: {len(df)} records")
print(f"Columns: {list(df.columns)}")
df.head()

Dataset loaded: 10000 records
Columns: ['order_id', 'user_id', 'user_name', 'city', 'membership', 'restaurant_id', 'restaurant_name_orders', 'cuisine', 'rating', 'order_date', 'total_amount']


,order_id,user_id,user_name,city,membership,restaurant_id,restaurant_name_orders,cuisine,rating,order_date,total_amount
0,1,2508,User_2508,Hyderabad,Regular,450,New Foods Chinese,Mexican,3.2,2023-02-18,842.97
1,2,2693,User_2693,Pune,Regular,309,Ruchi Curry House Multicuisine,Indian,4.5,2023-01-18,546.68
2,3,2084,User_2084,Chennai,Gold,107,Spice Kitchen Punjabi,Mexican,4.0,2023-07-15,163.93
3,4,319,User_319,Bangalore,Gold,224,Darbar Kitchen Non-Veg,Chinese,4.8,2023-10-04,1155.97
4,5,1064,User_1064,Pune,Regular,293,Royal Eatery South Indian,Italian,3.0,2023-12-25,1321.91


---
## Part 3: Multiple Choice Questions

In [23]:
# Q1: Which city has the highest total revenue (total_amount) from Gold members?
# Options: Hyderabad, Bangalore, Chennai, Pune

gold_revenue_by_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("Gold Member Revenue by City:")
print(gold_revenue_by_city)
print(f"\n>>> ANSWER: {gold_revenue_by_city.idxmax()}")

Gold Member Revenue by City:
city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

>>> ANSWER: Chennai


In [24]:
# Q2: Which cuisine has the highest average order value across all orders?
# Options: Indian, Chinese, Italian, Mexican

avg_order_by_cuisine = df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print("Average Order Value by Cuisine:")
print(avg_order_by_cuisine)
print(f"\n>>> ANSWER: {avg_order_by_cuisine.idxmax()}")

Average Order Value by Cuisine:
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

>>> ANSWER: Mexican


In [25]:
# Q3: How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
# Options: < 500, 500 – 1000, 1000 – 2000, > 2000

user_total_orders = df.groupby('user_id')['total_amount'].sum()
users_above_1000 = (user_total_orders > 1000).sum()
print(f"Distinct users with total orders > ₹1000: {users_above_1000}")

if users_above_1000 < 500:
    answer = '< 500'
elif users_above_1000 <= 1000:
    answer = '500 – 1000'
elif users_above_1000 <= 2000:
    answer = '1000 – 2000'
else:
    answer = '> 2000'
print(f"\n>>> ANSWER: {answer}")

Distinct users with total orders > ₹1000: 2544

>>> ANSWER: > 2000


In [26]:
# Q4: Which restaurant rating range generated the highest total revenue?
# Options: 3.0 – 3.5, 3.6 – 4.0, 4.1 – 4.5, 4.6 – 5.0

def rating_range(r):
    if r <= 3.5:
        return '3.0 – 3.5'
    elif r <= 4.0:
        return '3.6 – 4.0'
    elif r <= 4.5:
        return '4.1 – 4.5'
    else:
        return '4.6 – 5.0'

df['rating_range'] = df['rating'].apply(rating_range)
revenue_by_rating = df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)
print("Revenue by Rating Range:")
print(revenue_by_rating)
print(f"\n>>> ANSWER: {revenue_by_rating.idxmax()}")

Revenue by Rating Range:
rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

>>> ANSWER: 4.6 – 5.0


In [27]:
# Q5: Among Gold members, which city has the highest average order value?
# Options: Hyderabad, Bangalore, Chennai, Pune

gold_avg_by_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)
print("Gold Members - Average Order Value by City:")
print(gold_avg_by_city)
print(f"\n>>> ANSWER: {gold_avg_by_city.idxmax()}")

Gold Members - Average Order Value by City:
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

>>> ANSWER: Chennai


In [28]:
# Q6: Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
# Options: Indian, Chinese, Italian, Mexican

restaurants_by_cuisine = df.groupby('cuisine')['restaurant_id'].nunique()
revenue_by_cuisine = df.groupby('cuisine')['total_amount'].sum()
cuisine_stats = pd.DataFrame({
    'distinct_restaurants': restaurants_by_cuisine,
    'total_revenue': revenue_by_cuisine
}).sort_values('distinct_restaurants')
print("Cuisine Statistics:")
print(cuisine_stats)
print(f"\n>>> ANSWER: {cuisine_stats['distinct_restaurants'].idxmin()}")

Cuisine Statistics:
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09

>>> ANSWER: Chinese


In [29]:
# Q7: What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
# Options: 40%, 45%, 50%, 55%

total_orders = len(df)
gold_orders = len(df[df['membership'] == 'Gold'])
gold_percentage = (gold_orders / total_orders) * 100
print(f"Total orders: {total_orders}")
print(f"Gold member orders: {gold_orders}")
print(f"Percentage: {gold_percentage:.2f}%")
print(f"\n>>> ANSWER: {round(gold_percentage)}%")

Total orders: 10000
Gold member orders: 4987
Percentage: 49.87%

>>> ANSWER: 50%


In [30]:
# Q8: Which restaurant has the highest average order value but less than 20 total orders?
# Options: Grand Cafe Punjabi, Grand Restaurant South Indian, Ruchi Mess Multicuisine, Ruchi Foods Chinese

restaurant_stats = df.groupby('restaurant_name_orders').agg({
    'total_amount': ['mean', 'count']
}).reset_index()
restaurant_stats.columns = ['restaurant_name', 'avg_order', 'order_count']

options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
filtered = restaurant_stats[(restaurant_stats['order_count'] < 20) & (restaurant_stats['restaurant_name'].isin(options))]
filtered = filtered.sort_values('avg_order', ascending=False)

print("Restaurants with < 20 orders (from options):")
print(filtered)
print(f"\n>>> ANSWER: {filtered.iloc[0]['restaurant_name']}")

Restaurants with < 20 orders (from options):
         restaurant_name   avg_order  order_count
254  Ruchi Foods Chinese  686.603158           19

>>> ANSWER: Ruchi Foods Chinese


In [31]:
# Q9: Which combination contributes the highest revenue?
# Options: Gold + Indian, Gold + Italian, Regular + Indian, Regular + Chinese

combos = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
print("Revenue by Membership + Cuisine Combination:")
combo_revenues = {}
for m, c in combos:
    rev = df[(df['membership'] == m) & (df['cuisine'] == c)]['total_amount'].sum()
    combo_revenues[f"{m} + {c}"] = rev
    print(f"  {m} + {c}: ₹{rev:,.0f}")

best_combo = max(combo_revenues, key=combo_revenues.get)
print(f"\n>>> ANSWER: {best_combo} cuisine")

Revenue by Membership + Cuisine Combination:
  Gold + Indian: ₹979,312
  Gold + Italian: ₹1,005,779
  Regular + Indian: ₹992,100
  Regular + Chinese: ₹952,791

>>> ANSWER: Gold + Italian cuisine


In [32]:
# Q10: During which quarter of the year is the total revenue highest?
# Options: Q1 (Jan–Mar), Q2 (Apr–Jun), Q3 (Jul–Sep), Q4 (Oct–Dec)

df['quarter'] = df['order_date'].dt.quarter
revenue_by_quarter = df.groupby('quarter')['total_amount'].sum()
quarter_names = {1: 'Q1 (Jan–Mar)', 2: 'Q2 (Apr–Jun)', 3: 'Q3 (Jul–Sep)', 4: 'Q4 (Oct–Dec)'}

print("Revenue by Quarter:")
for q, rev in revenue_by_quarter.items():
    print(f"  {quarter_names[q]}: ₹{rev:,.2f}")

print(f"\n>>> ANSWER: {quarter_names[revenue_by_quarter.idxmax()]}")

Revenue by Quarter:
  Q1 (Jan–Mar): ₹2,010,626.64
  Q2 (Apr–Jun): ₹1,945,348.72
  Q3 (Jul–Sep): ₹2,037,385.10
  Q4 (Oct–Dec): ₹2,018,263.66

>>> ANSWER: Q3 (Jul–Sep)


---
## Part 4: Numerical Questions

In [33]:
# Q1: How many total orders were placed by users with Gold membership?

gold_orders = len(df[df['membership'] == 'Gold'])
print(f">>> ANSWER: {gold_orders}")

>>> ANSWER: 4987


In [34]:
# Q2: What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

hyderabad_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())
print(f">>> ANSWER: {hyderabad_revenue}")

>>> ANSWER: 1889367


In [35]:
# Q3: How many distinct users placed at least one order?

distinct_users = df['user_id'].nunique()
print(f">>> ANSWER: {distinct_users}")

>>> ANSWER: 2883


In [36]:
# Q4: What is the average order value (rounded to 2 decimals) for Gold members?

gold_avg = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)
print(f">>> ANSWER: {gold_avg}")

>>> ANSWER: 797.15


In [37]:
# Q5: How many orders were placed for restaurants with rating ≥ 4.5?

orders_high_rating = len(df[df['rating'] >= 4.5])
print(f">>> ANSWER: {orders_high_rating}")

>>> ANSWER: 3374


In [38]:
# Q6: How many orders were placed in the top revenue city among Gold members only?

top_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
orders_top_city_gold = len(df[(df['membership'] == 'Gold') & (df['city'] == top_city)])
print(f"Top revenue city for Gold members: {top_city}")
print(f">>> ANSWER: {orders_top_city_gold}")

Top revenue city for Gold members: Chennai
>>> ANSWER: 1337


---
## Part 5: Fill-in-the-Blank Questions

In [39]:
# Q1: The column used to join orders.csv and users.json is __________.

print(">>> ANSWER: user_id")

>>> ANSWER: user_id


In [40]:
# Q2: The dataset containing cuisine and rating information is stored in __________ format.

print(">>> ANSWER: SQL")

>>> ANSWER: SQL


In [41]:
# Q3: The total number of rows in the final merged dataset is __________.

total_rows = len(df)
print(f">>> ANSWER: {total_rows}")

>>> ANSWER: 10000


In [42]:
# Q4: If a user has no matching record in users.json, the merged values will be __________.

print(">>> ANSWER: NaN")

>>> ANSWER: NaN


In [43]:
# Q5: The Pandas function used to combine datasets based on a key is __________.

print(">>> ANSWER: merge")

>>> ANSWER: merge


In [44]:
# Q6: The column membership in the final dataset originates from the __________ file.

print(">>> ANSWER: users.json")

>>> ANSWER: users.json


In [45]:
# Q7: The join key used to combine orders data with restaurant details is __________.

print(">>> ANSWER: restaurant_id")

>>> ANSWER: restaurant_id


In [46]:
# Q8: The column that helps identify the type of food served by a restaurant is __________.

print(">>> ANSWER: cuisine")

>>> ANSWER: cuisine


In [47]:
# Q9: If a user places multiple orders, their personal details appear __________ times in the final merged dataset.

# Example verification
user_counts = df['user_id'].value_counts()
example_user = user_counts.idxmax()
print(f"Example: User {example_user} appears {user_counts[example_user]} times")
print(">>> ANSWER: multiple")

Example: User 2973 appears 13 times
>>> ANSWER: multiple


---
## Summary of All Answers

In [48]:
print("="*70)
print("MULTIPLE CHOICE ANSWERS")
print("="*70)
print("Q1: Chennai")
print("Q2: Mexican")
print("Q3: > 2000")
print("Q4: 4.6 – 5.0")
print("Q5: Chennai")
print("Q6: Chinese")
print("Q7: 50%")
print("Q8: Ruchi Foods Chinese")
print("Q9: Gold + Italian cuisine")
print("Q10: Q3 (Jul–Sep)")

print("\n" + "="*70)
print("NUMERICAL ANSWERS")
print("="*70)
print("Q1: 4987")
print("Q2: 1889367")
print("Q3: 2883")
print("Q4: 797.15")
print("Q5: 3374")
print("Q6: 1337")

print("\n" + "="*70)
print("FILL-IN-THE-BLANK ANSWERS")
print("="*70)
print("Q1: user_id")
print("Q2: SQL")
print("Q3: 10000")
print("Q4: NaN")
print("Q5: merge")
print("Q6: users.json")
print("Q7: restaurant_id")
print("Q8: cuisine")
print("Q9: multiple")

MULTIPLE CHOICE ANSWERS
Q1: Chennai
Q2: Mexican
Q3: > 2000
Q4: 4.6 – 5.0
Q5: Chennai
Q6: Chinese
Q7: 50%
Q8: Ruchi Foods Chinese
Q9: Gold + Italian cuisine
Q10: Q3 (Jul–Sep)

NUMERICAL ANSWERS
Q1: 4987
Q2: 1889367
Q3: 2883
Q4: 797.15
Q5: 3374
Q6: 1337

FILL-IN-THE-BLANK ANSWERS
Q1: user_id
Q2: SQL
Q3: 10000
Q4: NaN
Q5: merge
Q6: users.json
Q7: restaurant_id
Q8: cuisine
Q9: multiple
